In [18]:
import os
import pandas as pd

print(os.getcwd())



/Users/raniabenhamidane/Desktop/Israel-Palestine


In [19]:
df = pd.read_excel("disasters_raw_dataset.xlsx")
df.head()


,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,GADM Admin Units,Entry Date,Last Update
0,2002-0351-USA,No,nat-cli-wil-for,Natural,Climatological,Wildfire,Forest fire,NaN,NaN,USA,...,NaN,NaN,NaN,20000.0,34879.0,57.341840,"[{""adm1_code"":3219,""adm1_name"":""Colorado""}]","[{""gid_1"":""USA.6_1"",""migration_date"":""2025-12-...",2003-07-01,2025-12-20
1,2024-9796-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,USA,...,NaN,NaN,NaN,5400000.0,5400000.0,100.000000,NaN,"[{""gid_1"":""USA.13_1"",""name_1"":""Idaho""},{""gid_1...",2024-10-29,2025-12-20
2,2000-0035-USA,No,tec-tra-air-air,Technological,Transport,Air,Air,NaN,MD-83,USA,...,NaN,NaN,NaN,NaN,NaN,54.895152,NaN,NaN,2003-07-01,2023-09-25
3,2000-0262-USA,No,tec-tra-air-air,Technological,Transport,Air,Air,NaN,Twin engine turbo-prop BA-31 Jetstream,USA,...,NaN,NaN,NaN,NaN,NaN,54.895152,NaN,NaN,2003-07-01,2023-09-25
4,2000-0263-USA,No,tec-mis-col-col,Technological,Miscellaneous accident,Collapse (Miscellaneous),Collapse (Miscellaneous),NaN,Pedestrian bridge,USA,...,NaN,NaN,NaN,NaN,NaN,54.895152,NaN,NaN,2003-07-01,2025-03-14


In [20]:

df.columns


Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'GADM Admin Units', 'Entry Date', 'Last Update'],
      dtype='object')

In [144]:
#creating a copy of the dataset
df_work = df.copy()

df_work["start_date"] = pd.to_datetime(
    dict(
        year=df_work["Start Year"],
        month=df_work["Start Month"],
        day=df_work["Start Day"],
    ),
    errors="coerce",
)

# Check: combien de dates manquantes ?
df_work["start_date"].isna().mean()
#Check: quel type de disasters ont des dates manquantes
df_work[df_work["start_date"].isna()]["Disaster Type"].value_counts()

# Drop disasters with incomplete start dates (4.74%), as the analysis is conducted at the daily level
df_work = df_work[df_work["start_date"].notna()].copy()
df_work["start_date"].isna().mean()

np.float64(0.0)

In [145]:
#filter period from 2000-2011 to check 106 events like paper
df_work = df_work[
    (df_work["start_date"].dt.year >= 2000) &
    (df_work["start_date"].dt.year <= 2011)
].copy()

# Check
df_work["start_date"].dt.year.min(), df_work["start_date"].dt.year.max(), df_work.shape[0]





(np.int32(2000), np.int32(2011), 321)

In [146]:
#Filtre Natural & technological 
df_work = df_work[df_work["Disaster Group"].isin(["Natural", "Technological"])].copy()
df_work["Total Deaths"] = pd.to_numeric(df_work["Total Deaths"], errors="coerce")

# Keep only non-missing deaths (consistent with “high number of victims”)
df_work = df_work[df_work["Total Deaths"].notna()].copy()

# Check summary
df_work["Total Deaths"].describe()

count     245.000000
mean       25.722449
std       121.432453
min         1.000000
25%         4.000000
50%        10.000000
75%        19.000000
max      1833.000000
Name: Total Deaths, dtype: float64

In [147]:
median_by_type = (
    df_work.groupby("Disaster Type")["Total Deaths"]
    .median()
    .rename("median_deaths")
    .reset_index()
)


In [148]:
#compute median for number of death by disaster
median_by_type = (
    df_work.groupby("Disaster Type")["Total Deaths"]
    .median()
    .rename("median_deaths")
    .reset_index()
)
#check 
median_by_type.sort_values("median_deaths", ascending=False).head(20)


,Disaster Type,median_deaths
3,Epidemic,214.0
6,Extreme temperature,28.5
10,Miscellaneous accident (General),21.0
0,Air,19.0
9,Mass movement (wet),15.0
14,Water,14.5
4,Explosion (Industrial),13.0
12,Road,13.0
1,Collapse (Miscellaneous),11.5
7,Fire (Miscellaneous),10.0


In [149]:
#associate each median to its disaster type
df_work = df_work.merge(median_by_type, on="Disaster Type", how="left")

#keep only the half that killed at least as much as the median
df_work = df_work[df_work["Total Deaths"] >= df_work["median_deaths"]].copy()

#how many disasters left
df_work.shape

#The final number of disasters differs from the original paper due to differences in the underlying data.


(135, 49)